# Week Five: Integration and differentiation

To be graded, your notebook must be runnable start to finish. If you can't make an in-notebook test pass, comment it out for to attempt to get partial credit. You should replace the `...` markers with your code. Do not change the names of the pre-defined ALL_CAPS variables and functions. (If you start from scratch, make sure you match the requested function names and requested ALL_CAPS variables). Other that that, you are free to define or make anything you wish. Remember that functions will often be tested with different numbers than the ones you are given.

In [ ]:
# EID is your 6+2 UC Electronic ID
EID = '800245992'
NAME = 'Elias Garden'

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Problem 1: 2D Trapezoidal rule

You can extend the trapezoidal rule to 2D. I'll give you a few hints:

The weight matrix is:
$$
W_{ij} = \begin{matrix}
1 & 2 & 2 & \cdots & 2 & 2 & 1 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
1 & 2 & 2 & \cdots & 2 & 2 & 1
\end{matrix}
$$

If you use the trick I used in the slides instead of explicitly creating this matrix, you would split the sum into 4 parts. However, it's really quite easy to make the matrix above

The "bin" width $h$ is joined by the 2nd dimension bin width of $k$, and then the trapezoidal rule integral value is 

$$
I_\mathrm{tr} = \frac{1}{4} h k \sum_i \sum_j W_{ij} f(x_{i}, y_{j})
$$

Write a function that takes a 2-parameter function 'f' and integrates it with the given parameters (further explanations are built into the docstring of the function). I'm writing a Numpy-style docstring.

See also: <http://mathfaculty.fullerton.edu/mathews/n2003/SimpsonsRule2DMod.html>

In [ ]:
def trap_2d(f, xrange, yrange, xN, yN):
    '''Integrate a 2D function with the trapezoidal rule.
    
    This takes a 2D function and integrates it over a 2D grid.
    This uses the Trapezoidal rule in 2D.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take an x and y value (possibly arrays).
    xrange : (float, float)
        The min and max values for x, as an array
    yrange : (float, float)
        The min and max values for y, as an array
    xN : int
        The number of evaluations along x
    yN : int
        The number of evaluations along x

    Returns
    -------
    float
        The total integral
    
    '''
    # If these are *not* arrays, convert them into arrays. Otherwise, leave them alone.
    # (If we used np.array, we'd needlessly make a copy if it already was an array)
    xrange = np.asarray(xrange)
    yrange = np.asarray(yrange)

    # There are N-1 "bins"
    x_iter = (xrange[1] - xrange[0])/(xN-1)
    y_iter = (yrange[1] - yrange[0])/(yN-1)

    #these create the arrays that contain the values along the axis
    x = np.zeros(xN)
    for i in range(0, xN):
        x[i] = xrange[0] + (x_iter * i)

    y = np.zeros(yN)
    for i in range(0, yN):
        y[i] = yrange[0] + (y_iter * i)

    #this creates a double array that of all the values fxy
    f_xy = np.zeros([xN, yN])
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            f_xy[i][j] = f(xi,yj)

    #this creates the weight matrix
    w = np.zeros([xN, yN])
    for i in range(0, xN):
	    for j in range(0, yN):
		    if ((i == 0) and (j == 0)) or ((i == 0) and (j == yN-1)) or ((i == xN-1) and (j == 0)) or ((i == xN-1) and (j == yN-1)):
		    	w[i][j] = 1
		    elif (i == 0) or (i == xN-1) or (j == 0) or (j == yN-1):
		    	w[i][j] = 2
		    else:
		    	w[i][j] = 4

    #summing for-loop
    sums = 0.0
    for i in range(0, xN):
        for j in range(0, yN):
            sums += w[i][j] * f_xy[i][j]
    area_approx = (1/4) * x_iter * y_iter * sums
    return area_approx

Feel free to add more tests, I'm adding the simplest one I can think of:

In [ ]:
def f(x,y):
    return 1

In [ ]:
area = trap_2d(f, [0, 10], [0, 10], 10, 10)
assert np.isclose(area, 100)

# Problem 2: MC integration

Write a function that performs MC integration of an unknown function of N dimensions. You can use `f(*array)` to call the function with the correct number of arguments (it expands to `f(array[0], array[1], ..., array[N])`). Return **two** values; the estimated integral and the estimated error on the integral.

In [ ]:
def mc_integrate(f, N, samples, ranges):
    '''
    Integrate an ND function with a set of sampled MC points.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take N
        values (possibly equal length arrays).
    N : int
        The dimensionality of the function
    samples : int
        The number of samples to integrate over
    ranges : array[N, 2]
        An array with the min and max for each dimension.

    Returns
    -------
    float
        The total integral
    float
        The estimated error = relative error * the total integral.
    '''

    ranges = np.asarray(ranges) # Just making sure ranges is an array

    #first we need to get our random array and adjust it to the correct size
    rand_array = []
    for i in range(N):
        sample_to_append = np.random.rand(samples)
        sample_to_append *= ranges[i][1] - ranges[i][0]
        sample_to_append += ranges[i][0]
        rand_array.append(sample_to_append)

    #this transforms this to an array of points
    rand_array = np.transpose(rand_array)

    #quickly calculate an approximation of the max value
    lin_array = []
    for i in range(N):
        lin_array.append(np.linspace(ranges[i][0], ranges[i][1], num = samples))
    lin_array = np.transpose(lin_array)
    max_val = 0
    for i in lin_array:
        if f(*i) > abs(max_val):
            max_val = f(*i)
    #max_val will be useful for calculating the ratios for the area

    #now we must create the f(N) possible values. this is done by creating an array
    #that will iterate evenly to the max value we established earlier
    #these are the values that we will check to see if they are under the curve
    f_n_vals = np.linspace(0, max_val, samples)

    #now iterate over all the points to see which are under the curve
    under_curve = []
    for i in range(len(rand_array)):
        if f_n_vals[i] < f(*rand_array[i]):
            under_curve.append(i)

    #now calculate the ratio of under the curve to total then compare that with original
    point_ratio = len(under_curve) / len(rand_array)
    dim = 1
    for i in ranges:
        dim *= i[1] - i[0]
    tot_area = dim * max_val

    #find error
    mc_error = 1 / np.sqrt(N)

    return tot_area * point_ratio, mc_error

In [ ]:
def f(x,y):
    return x + y

In [ ]:
integral, error = mc_integrate(f, 2, 10_000, [[0,1],[0,1]])

In [ ]:
print(integral, error)

#### For comparison, we could use NQuad from Scipy:

In [ ]:
from scipy import integrate

In [ ]:
integrate.nquad(f, [[0,1],[0,1]])

# Problem 3: Second derivatives

Write a function that calculates:

$$
\frac{d^2 f(t)}{dt^2} \Biggr\rvert_\mathrm{cd}
=
\frac{
\left[
f(t + h) - f(t)
\right] - \left[
f(t) - f(t - h)
\right]
}{h^2}
$$

(The notes in class had a typo, fixed here.) Write a nice docstring as well.

In [ ]:
def d2fdt2(f, t, h):
    '''
    To calculate the second derivative for a particular value

    Parameters:
    __________
        f: the function to be differentiated
        t: the input to the function f
        h: the width of the difference

    Returns:
    ________
    float:
        the value for f''(t)
    '''
    f_twoprime = ((f(t + h) - f(t)) - (f(t) - f(t - h))) / (h ** 2)
    return f_twoprime

Test your `d2fdt2` function on a function that you know the derivative for.

In [ ]:
print('my function:', d2fdt2(np.sin, 5, 1e-5))
print('actual -sin:', (-1 * np.sin(5)))
assert np.isclose(d2fdt2(np.sin, 5, 1e-5), (-1 * np.sin(5)))